[Pocket is shutting down](https://support.mozilla.org/en-US/kb/future-of-pocket)

I don't trust that anyone else who has alternatives is going to be able to necessarily be around forever so I guess I have to make it myself.

# Requirements
So here's what I want

## Goals
- A browser extension that I can trigger on any link
- An Android app that I can share links to
- The ability to tag links
- The ability to search stored links searching across both tags and text
- Keep costs low. We can use AWS or something, but I want to stay within the free tier as much as possible
- Some sort of monitoring

## Nice-to-haves
- Store the full article text
- Ability to pierce paywalls somewhat
- Use the archive link if the full link has a paywall
- AI-enabled search via embeddings
- CI/CD

## Non-goals
- Dealing with the app store, I'll side-load the app if need be
- Multi-tenancy, I'm ok with just a single account for the entire instance
- Scale


# Setup

## Project Setup
To set up the development environment:

This project uses [uv](https://docs.astral.sh/uv/getting-started/installation/) for managing all things Python including notebooks. This is largely to run tests and notebooks locally. Start the juypter server from this folder

## AWS Account Configuration

### SSO
Optional but recommended: follow the steps in the jupyter notebook here to set it up [here.](./aws-configuration.ipynb)

# Architecture

## V0

Here's how this is going to work:

### User document store interface

A chrome plugin will - when triggered - give an optional popup to prefill tags, then  take that and the full content of the current web page as html or text, the url, and sends it (and maybe some images) to our "index-document" web service.

### AWS Infrastructure

I want to support multiple environments so I will use a suffix for this in my resources. Also all resources should be tagged `just-my-links: environment-name`. Below when I say `dev` I really mean whatever environment is selected.

Note that we should not use a vpc as we're trying to keep costs to virtually zero, we would need a  vpc endpoint for our lambda to communicate with secret manager and vpc endpoints cost $8/mo

All operations in s3 will be within a single bucket `just-my-links--dev`

I don't want to pay for a server and this is low use so this is something that is implemented via an api gateway http endpoint that triggers aws lambda. The lambda will be named `just-my-links--store-document--dev` and the implementation should be in `python3.13`, and deployed via a container that is stored in an ecr registry (`just-my-links--ecr-store-document--dev`).

This service implementation will use aws lambda powertools for their good logging defaults, typing, api handling structures, and so on

Logs need to be written to s3 for backup. There should be alerts on that bucket filling up to be too large that email me.

Within that service we are going to be receiving a multipart formdata request and a document_url that is part of our querystring. we will write it to a document-storage folder in s3 under a key that is the hash of the document_url. All files will be stored under their multipart form data filename. The first one must be named document.html or document.txt. We will then write a .metadata.json file that will contain the documentUrl and the "entrypoint" (which will be `document.txt` or `document.html`).

Finally, the lambda will broadcast an event via eventbridge (named `just-my-links--events--dev`) with a `type: "Document stored"`. Which contains the folderPath in s3 and the documentUrl.

Only one of these lambdas should run at a time. The lambda should have authentication though all it's going to do is safely compare a submitted bearer token against an auth token stored in secret manager

An eventbridge rule will configure events of this type to pass into an sqs queue `just-my-links--index-documents-trigger--dev`. This will trigger a separate lambda named `just-my-links--index-documents` (again, concurrency 1). Do not use async invocation here, just wire up the sqs queue to trigger the lambda. Also create a dead letter queue (`just-my-links--index-documents-dlq--dev`) and have a montior that sends emails when something enters that queue.

Our `just-my-links--index-documents`  will copy our chromadb file from s3 (if it exists) into the lambda's /tmp directory and use the chromadb python package to open it. For each received event it will read the event's folder and read in its .metadata.json. It will then upsert an array under the key of the documentUrl. The array will contain the contents of entrypoint as the first value and any other txt or html file contents as the remainder. Broadcast an event to our eventbridge bus `type: "Document indexed"` with the `folderPath` and `documentUrl`. Then the chromadb file should be copied back to s3.

### User query interface

For v1 we will create a command line tool - it can be a python script. This will copy the chromadb over to a local tmp directory and allow querying of it. Using this I should be able to query my chromadb store using natural language and get documentUrls.

This command line should have an option where rather than copying chromadb over to local, it uses a file in a provided location


### Data Flow:

system has a two-stage processing pipeline:
- Stage 1 - Document Storage:
  - Chrome plugin sends multipart form data + document_url to API Gateway
  - store-document Lambda authenticates via Secrets Manager
  - Lambda stores files in S3 under hashed folder structure with metadata
  - Lambda publishes "Document stored" event to EventBridge
  - EventBridge rule forwards event to SQS queue
- Stage 2 - Document Indexing:
  - SQS triggers index-documents Lambda
  - Lambda downloads ChromaDB from S3 to /tmp
  - Lambda reads document files and metadata from S3
  - Lambda upserts document content array into ChromaDB
  - Lambda uploads updated ChromaDB back to S3
  - Lambda publishes "Document indexed" event

- Query Interface:
  - CLI tool downloads ChromaDB locally for natural language queries
  - Returns document URLs based on semantic search